### Шаг 1. Откройте файлы с данными

In [ ]:
# подключаем зависимости
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import phik
from phik.report import plot_correlation_matrix

from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    LabelEncoder
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from catboost import cv
from catboost import CatBoostClassifier, Pool

from imblearn.over_sampling import SMOTE

import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
data_train = pd.read_csv('/kaggle/input/music-genre/kaggle_music_genre_train.csv')
data_train.head(10)

In [ ]:
data_train.info()

Видно, что есть null значения в столбцах key, mode и tempo.

In [ ]:
data_test = pd.read_csv('/kaggle/input/music-genre/kaggle_music_genre_test.csv')
data_test.head(10)

In [ ]:
data_test.info()

В тестовой выборке также есть null значения в столбцах key, mode и tempo.

### Шаг 2. Предобработка и исследовательский анализ данных

Кроме истинных null значений, в описании датасета в Kaggle видно, что duration_ms может приинмать значение -1. Для упрощения работы далее, можем интерпретировать эти данные как null.

In [ ]:
for data in [data_train, data_test]:
    data.loc[data['duration_ms'] == -1, 'duration_ms'] = None

Проверим что null встали на свои места.

In [ ]:
for data in [data_train, data_test]:
    print(data.isnull().sum())
    print()

Проверим явные на дубликаты

In [ ]:
for data in [data_train, data_test]:
    print(data.duplicated().sum())

Явных дубликатов нет, поэтому нужно проверить на неявные дубликаты. С учетом того, что есть null данные, будем дубликаты по косвенным признакам, сверяя данные в столбцах, в которых нет пропусков

In [ ]:
duplicated_features_subset = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'music_genre']
duplicated_rows = data_train[data_train.duplicated(subset=duplicated_features_subset, keep=False)]
duplicated_rows.sort_values(by=duplicated_features_subset).head()

Проанализируем дубликаты на пропуски, чтобы по возможности их обработать.

In [ ]:
duplicated_rows.isnull().sum()

Посмотрим что за стоки с пропущенным key

In [ ]:
duplicated_rows[duplicated_rows['key'].isnull()]

key пропущен в обоих записях. удалим дубли

In [ ]:
data_train = data_train.drop_duplicates(subset=duplicated_features_subset, ignore_index=True)
data_train.info()

7 дубликатов удалили. Теперь проверим, есть ли такие дубликаты, у которых разные жанры.

In [ ]:
duplicated_features_subset.pop()

In [ ]:
duplicated_rows_train = data_train[data_train.duplicated(subset=duplicated_features_subset, keep=False)]
duplicated_rows_train.sort_values(by=duplicated_features_subset).head()

In [ ]:
duplicated_rows_train['instance_id'].count()

Заодно посмотрим на дубликаты в тестовой выборке

In [ ]:
duplicated_rows_test = data_test[data_test.duplicated(subset=duplicated_features_subset, keep=False)]
duplicated_rows_test.sort_values(by=duplicated_features_subset).head()

In [ ]:
duplicated_rows_test['instance_id'].count()

Выводы которые можно сделать:
- т.к. в обучающей выборе дубликаты идентифицируются разными жанрами - это будет мешать обучению моделей - нам придется как-то эту ситуацию обработать.
- т.к. в тестовой выборке тоже есть дубликаты - мы можем использовать информацию о дубликатах в обоих выборках для того, чтобы попробовать заполнить null-значения.

In [ ]:
duplicated_rows_test.isnull().sum()

In [ ]:
duplicated_rows_test[duplicated_rows_test.isnull().any(axis=1)].sort_values(by=duplicated_features_subset)

Действительно можно заполнить пропуски по тем данным, что у нас есть в дубликатах. Чтобы заполнить пропуски, нам нужно однозначно идентифицировать треки. Введем для этого признак hash, рассчитанный на базе колонок, в которых нет пропусков.

In [ ]:
def get_hash(row):
   return hash(tuple(row[duplicated_features_subset]))

data_train['hash'] = data_train.apply(get_hash, axis=1)
data_test['hash'] = data_test.apply(get_hash, axis=1)

In [ ]:
data_train.head()

Объединим все данные в один большой справочник

In [ ]:
data_all = pd.concat([data_train, data_test], axis=0, join='outer').reset_index()

Произведем поиск пропущенных значений среди всех данных которые у нас есть.

In [ ]:
nullable_features = ['duration_ms', 'mode', 'key', 'tempo']
for data in [data_train, data_test]:
    for nullable_feature in nullable_features:
        hash_list = data[data[nullable_feature].isnull()]['hash']
        for hash_value in hash_list:
            values = data_all[(data_all['hash'] == hash_value)&(~data_all[nullable_feature].isnull())]
            if(not values.empty):
                value = values.iloc[0][nullable_feature]
                data.loc[(data['hash'] == hash_value)&(data[nullable_feature].isnull()), nullable_feature] = value

Удалим дубликаты из обучающей выборки. Оставим только первый жанр. Будем считать, что он основной.

In [ ]:
data_train = data_train.drop_duplicates(subset=duplicated_features_subset, keep='first', ignore_index=True)

Посмотрим сколько осталось null значений.

In [ ]:
for data in [data_train, data_test]:
    print(data.isnull().sum())
    print()

Удалим колонки, которые не будем использовать в анализе

In [ ]:
features_train = data_train.drop(['instance_id', 'track_name', 'obtained_date', 'hash'], axis=1)
features_test = data_test.drop(['instance_id', 'track_name', 'obtained_date', 'hash'], axis=1)

In [ ]:
features_train.info()

In [ ]:
features_test.info()

tempo является числовым признаком, чтобы обработать пропуски - заменим их на медиану в каждом жанре отдельно. в тестовой выборке на общую медиану.

Прежде чем рассчитывать медиану, посмотрим на адекватность данных

In [ ]:
for data in [features_train, features_test]:
    print(data['tempo'].describe())
    print()

Данные в порядке.

In [ ]:
genre_tempo_median = features_train.groupby(by='music_genre')['tempo'].median().reset_index(name='median')
genre_tempo_median

Прием для рассчета медианы по жанру мы будем использовать несколько раз, поэтому объявим функцию.

In [ ]:
def fill_median(f_train, f_test, f_name, genre_medians):
    for i, row in genre_medians.iterrows():
        selector = (f_train['music_genre'] == row['music_genre'])&(f_train[f_name].isnull())
        f_train.loc[selector, f_name] = row['median']
    
    f_test.loc[f_test[f_name].isnull(), f_name] = f_test[f_name].median()

In [ ]:
fill_median(features_train, features_test, 'tempo', genre_tempo_median)

In [ ]:
for data in [features_train, features_test]:
    print(data.isnull().sum())
    print()

Теперь этот же прием применим к длительности.

In [ ]:
genre_duration_median = features_train.groupby(by='music_genre')['duration_ms'].median().reset_index(name='median')
fill_median(features_train, features_test, 'duration_ms', genre_duration_median)

In [ ]:
for data in [features_train, features_test]:
    print(data.isnull().sum())
    print()

Стоит проверить категориальные признаки на уникальность.

In [ ]:
for data in [features_train, features_test]:
    print('key:')
    print(data['key'].sort_values().unique())
    print('mode:')
    print(data['mode'].sort_values().unique())

In [ ]:
features_train.pivot_table(index="music_genre", columns="mode", values='acousticness', aggfunc="count")

Во всех жанрах мажор встречается чаще, чем минор. Заполним пропуски по самой часто встречающейся моде.

In [ ]:
for data in [features_train, features_test]:
    mode = data['mode'].mode()[0];
    data['mode'] = data['mode'].fillna(mode)

In [ ]:
features_train.pivot_table(index="music_genre", columns="key", values='acousticness', aggfunc="count")

По этой таблице сложно сделать вывод. Во всех жанрах присутствуют все ключевые ноты. Заполним пропуски заглушкой

In [ ]:
for data in [features_train, features_test]:
    key = data['key'].mode()[0];
    data['key'] = data['key'].fillna('unknown')

In [ ]:
for data in [features_train, features_test]:
    print(data.isnull().sum())
    print()

Разобрались с null значениями.

**Проанализируем числовые признаки на адекватность значений.**

In [ ]:
for data in [features_train, features_test]:
    print(data['acousticness'].describe())
    print()

Нужно посмотреть что за 0 в обучающей выборке.

In [ ]:
for data in [features_train, features_test]:
    data['acousticness'].hist()

In [ ]:
for data in [features_train, features_test]:
    print(data[data['acousticness'] == 0]['acousticness'].count())

Скорей всего это реальное значение, но если нет - не сильно будет влиять на обучение.

In [ ]:
for data in [features_train, features_test]:
    print(data['danceability'].describe())
    print()

Здесь все адекватно

In [ ]:
for data in [features_train, features_test]:
    print(data['duration_ms'].describe())
    print()

Для удобства сразу переведу в секунды

In [ ]:
features_train['duration_sec'] = features_train['duration_ms'] / 1000
features_test['duration_sec'] = features_test['duration_ms'] / 1000
features_train = features_train.drop('duration_ms', axis=1)
features_test = features_test.drop('duration_ms', axis=1)

In [ ]:
for data in [features_train, features_test]:
    print(data['duration_sec'].describe())
    print()

Все адекватно.

In [ ]:
for data in [features_train, features_test]:
    print(data['energy'].describe())
    print()

Все адекватно.

In [ ]:
for data in [features_train, features_test]:
    print(data['instrumentalness'].describe())
    print()

Посмотрим на 0

In [ ]:
for data in [features_train, features_test]:
    data['instrumentalness'].hist()

In [ ]:
for data in [features_train, features_test]:
    print(data[data['instrumentalness'] == 0]['instrumentalness'].count())

Слишком большое количество имеет значение 0. Проверим распределение по жанрам.

In [ ]:
features_train[features_train['instrumentalness'] == 0].groupby(by='music_genre')['instrumentalness'].count()

Сложно сделать какое-то предположение. Но как будто бы получается, что и классическая музыка бывает со словами (Classical = 49). Принимаю решение заменить 0 на медиану по жанрам.

In [ ]:
features_train.loc[features_train['instrumentalness'] == 0, 'instrumentalness'] = None
features_test.loc[features_test['instrumentalness'] == 0, 'instrumentalness'] = None
genre_instrumentalness_median = features_train.groupby(by='music_genre')['instrumentalness'].median().reset_index(name='median')
fill_median(features_train, features_test, 'instrumentalness', genre_instrumentalness_median)

In [ ]:
for data in [features_train, features_test]:
    data['instrumentalness'].hist()

In [ ]:
for data in [features_train, features_test]:
    print(data['liveness'].describe())
    print()

Все адекватно.

In [ ]:
for data in [features_train, features_test]:
    print(data['loudness'].describe())

Все адекватно.

In [ ]:
for data in [features_train, features_test]:
    print(data['speechiness'].describe())
    print()

Все адекватно.

In [ ]:
for data in [features_train, features_test]:
    print(data['valence'].describe())
    print()

Посмотрим на 0

In [ ]:
for data in [features_train, features_test]:
    data['valence'].hist()

In [ ]:
for data in [features_train, features_test]:
    print(data[data['valence'] == 0]['valence'].count())

Оставляем.

Резюме. Разобрались с null значениями, проверили входные признаки на адекватность значений.

**Вводим новые признаки.**

В описании данных сказано как интерпретировать признаки liveness и speechiness.
Заменим их на категориальные.

In [ ]:
def get_liveness(row):
    l = row['liveness']
    if l > 0.8:
        return 'live'
    else:
        return 'recorded'

features_train['liveness'] = features_train.apply(get_liveness, axis=1)
features_test['liveness'] = features_test.apply(get_liveness, axis=1)

In [ ]:
def get_speechiness(row):
    l = row['speechiness']
    if l > 0.66:
        return 'vocal'
    elif l < 0.33:
        return 'musical'
    else:
        return 'mixed'

features_train['speechiness'] = features_train.apply(get_speechiness, axis=1)
features_test['speechiness'] = features_test.apply(get_speechiness, axis=1)

**Проверка на мультиколлинеарность.**

In [ ]:
interval_cols = ['acousticness', 'danceability', 'duration_sec', 'energy', 'instrumentalness', 'loudness', 'tempo', 'valence']
phik_overview = features_train.phik_matrix(interval_cols=interval_cols)
plot_correlation_matrix(phik_overview.values, 
                        x_labels=phik_overview.columns, 
                        y_labels=phik_overview.index, 
                        vmin=0, vmax=1, color_map="Blues", 
                        title=r"Корреляция $\phi_K$", 
                        fontsize_factor=1.5, 
                        figsize=(10, 8))
plt.tight_layout()

Нашел на википедии https://ru.wikipedia.org/wiki/%D0%9C%D1%83%D0%BB%D1%8C%D1%82%D0%B8%D0%BA%D0%BE%D0%BB%D0%BB%D0%B8%D0%BD%D0%B5%D0%B0%D1%80%D0%BD%D0%BE%D1%81%D1%82%D1%8C "Для обнаружения мультиколлинеарности факторов можно проанализировать непосредственно корреляционную матрицу факторов. Уже наличие больших по модулю (выше 0,7-0,8) значений коэффициентов парной корреляции свидетельствует о возможных проблемах с качеством получаемых оценок."
Примем, что коэффициент должен быть не выше 0.7

In [ ]:
CORRELATION_LIMIT = 0.75
for i, row in phik_overview.iterrows():
    for j, value in row.items():
        if(i > j and np.abs(value) > CORRELATION_LIMIT):
            print(i, j, value)

energy коррелирует с acousticness и loudness

При оценке корреляций мы должны обращать внимание не только на коэффициенты, но и на их статистическую значимость. Потому что, в конце концов, большая корреляция может оказаться статистически незначимой, и наоборот.

In [ ]:
significance_overview = features_train.significance_matrix(interval_cols=interval_cols)
plot_correlation_matrix(significance_overview.values, 
                        x_labels=significance_overview.columns, 
                        y_labels=significance_overview.index, 
                        vmin=-100, vmax=100, title="Значимость коэффициентов", 
                        color_map="Blues",
                        usetex=False, fontsize_factor=1.5, figsize=(14, 10))
plt.tight_layout()

In [ ]:
global_correlation, global_labels = features_train.global_phik(interval_cols=interval_cols)

plot_correlation_matrix(global_correlation, 
                        x_labels=[''], y_labels=global_labels, 
                        vmin=0, vmax=1, figsize=(3.5,4),
                        color_map="Blues", title=r"$g_k$",
                        fontsize_factor=1.5)
plt.tight_layout()

Больше всех со всеми остальными признаками коррелирует energy. Его оставим, а acousticness и loudness можно удалять

In [ ]:
features_train = features_train.drop(['acousticness', 'loudness'], axis=1)
features_test = features_test.drop(['acousticness', 'loudness'], axis=1)
interval_cols.remove('acousticness')
interval_cols.remove('loudness')

Разборались с мультиколлинеарностью.

выделим из features_train жанры в целевой признак

In [ ]:
target_train = features_train['music_genre']
features_train = features_train.drop(['music_genre'], axis=1)

Проверим целевой класс на дисбаланс

In [ ]:
music_genre = target_train.value_counts(normalize=True)
print(music_genre)
music_genre.plot(kind='bar', title='дисбаланс классов', xlabel='классы', ylabel='доля класса')
plt.show();

 Закодируем категориальные признаки и решим проблему дисбаланса.

### Шаг 3. Кодирование и масштабирование признаков

In [ ]:
#категориальные признаки для OHE
ohe_features = ['mode', 'key', 'liveness', 'speechiness']

# drop='first' удаляет первый признак из закодированных:
# таким образом обходим dummy-ловушку
# задаём handle_unknown='ignore':
# игнорируется ранее невстречающиеся значения признака (при transform)
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

# обучаем энкодер на заданных категориальных признаках тренировочной выборки
encoder_ohe.fit(features_train[ohe_features])
# encoder_ohe.get_feature_names_out() позволяет получить названия колонок
features_train[encoder_ohe.get_feature_names_out()] = \
    encoder_ohe.transform(features_train[ohe_features])

# удаляем незакодированные категориальные признаки (изначальные колонки)
features_train = features_train.drop(ohe_features, axis=1)
features_train.head()

In [ ]:
features_test[encoder_ohe.get_feature_names_out()] = \
    encoder_ohe.transform(features_test[ohe_features])

features_test = features_test.drop(ohe_features, axis=1)
features_test.head()

боремся с дисбалансом методом SMOTE

In [ ]:
smote = SMOTE(random_state=12345)
features_train, target_train = smote.fit_resample(features_train, target_train)

In [ ]:
music_genre = target_train.value_counts(normalize=True)
print(music_genre)
music_genre.plot(kind='bar', title='дисбаланс классов', xlabel='классы', ylabel='доля класса')
plt.show();

Смасштабируем числовые признаки. Нормализовывать будем те, которые не находятся в диапазоне от 0 до 1

In [ ]:
scaler = StandardScaler()

numeric_features = interval_cols
features_train[numeric_features] = scaler.fit_transform(features_train[numeric_features])
features_test[numeric_features] = scaler.transform(features_test[numeric_features])

In [ ]:
features_train.head()

Теперь закодируем целевой признак

In [ ]:
label_encoder = LabelEncoder()
target_train = label_encoder.fit_transform(target_train)
target_train

### Шаг 4. Разработка модели ML

In [ ]:
def get_score(model, features, target):
    scores = cross_val_score(model, features, target, scoring="f1_micro", cv=5) 
    return sum(scores) / len(scores)

In [ ]:
def get_optimal_decision_tree_classifier_description(features_train, target_train):
    best_score = 0
    best_depth = 0
    best_min_samples_leaf = 0
    for min_samples_leaf in range(1, 6):
        for depth in range(1,11):
            model = DecisionTreeClassifier(
                random_state=12345,
                max_depth=depth,
                min_samples_leaf=min_samples_leaf)
            score = get_score(model, features_train, target_train)
            if (score > best_score):
                best_score = score
                best_depth = depth
                best_min_samples_leaf = min_samples_leaf
    return {
        'score': best_score,
        'params': {
            'max_depth': best_depth,
            'min_samples_leaf': min_samples_leaf
        }
    }

In [ ]:
def get_optimal_logistic_regression_description(features_train, target_train):
    model = LogisticRegression(
        multi_class='multinomial',
        max_iter=1000,
        random_state=12345)
    return {
        'score': get_score(model, features_train, target_train),
        'params': None
    }

In [ ]:
def get_optimal_random_forest_classifier_description(features_train, target_train):
    best_score = 0
    best_est = 0
    best_depth = 0
    best_min_samples_leaf = 0
    #не успеваю сделать поиск параметров, т.к. делаю работу поздно вечером
    #а отчет отправить надо
    #поэтому намеренно сужаю диапазон поиска
    for min_samples_leaf in range(5, 6, 1):
        for est in range(50, 71, 10):
            for depth in range (8, 12, 2):
                model = RandomForestClassifier(
                    random_state=12345, 
                    n_estimators=est, 
                    max_depth=depth, 
                    min_samples_leaf=min_samples_leaf)
                
                score = get_score(model, features_train, target_train)
                if (score > best_score):
                    best_score = score
                    best_est = est
                    best_depth = depth
                    best_min_samples_leaf = min_samples_leaf
    return {
        'score': best_score,
        'params': {
            'n_estimators': best_est, 
            'max_depth': best_depth,
            'min_samples_leaf': best_min_samples_leaf
        }
    }

In [ ]:
def get_optimal_catboost_classifier_description(features_train, target_train):
    train_pool = Pool(features_train, target_train)
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 250, 500),
            'max_depth': trial.suggest_int('max_depth', 2, 4),
            'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 2.5, 4),
            'random_strength': trial.suggest_float('random_strength', 0.9, 1.4),
            'learning_rate': trial.suggest_float("eta", 1e-2, 1e-1, log=True),
            'min_data_in_leaf': trial.suggest_int("min_data_in_leaf", 1, 5),
            'random_state': trial.suggest_categorical('random_state', [12345]),
            'eval_metric': trial.suggest_categorical('eval_metric', ['TotalF1:average=Micro']),
            "loss_function": trial.suggest_categorical('loss_function', ['MultiClass']),
            'logging_level': trial.suggest_categorical('logging_level', ['Silent']),
            }

        scores = cv(train_pool, params, fold_count=5)
        return scores['test-TotalF1:average=Micro-mean'].values[-1]

    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=1e6, timeout=60*1)
    return {
        'score': study.best_trial.value,
        'params': study.best_trial.params
    }

In [ ]:
def analisys(features_train, target_train):
    return [
        {
            'Тип': 'Логистическая регрессия',
            **get_optimal_logistic_regression_description(features_train, target_train)
        },
        {
            'Тип': 'Решающее дерево',
            **get_optimal_decision_tree_classifier_description(features_train, target_train)
        },
        {
            'Тип': 'Случайный лес',
            **get_optimal_random_forest_classifier_description(features_train, target_train)
        },
        {
            'Тип': 'Catboost',
            **get_optimal_catboost_classifier_description(features_train, target_train)
        }
    ]

In [ ]:
def make_table(data):
    df = pd.DataFrame(data = data);
    with pd.option_context('display.precision', 3):
        style = df.style.background_gradient('coolwarm')
    return style

In [ ]:
# сохраним в перееменную, для итогового анализа
result = analisys(features_train, target_train)

In [ ]:
make_table(result)

Лучший результат у catboost: 0.649981

In [ ]:
params = {'n_estimators': 482, 'max_depth': 4, 'l2_leaf_reg': 3.303532893177284, 'random_strength': 1.0685831400944628, 'eta': 0.06360183429810953, 'min_data_in_leaf': 2, 'random_state': 12345, 'eval_metric': 'TotalF1:average=Micro', 'loss_function': 'MultiClass', 'logging_level': 'Silent'}
model = CatBoostClassifier(
    **params
)

model.fit(
    features_train, target_train,
    verbose=False
)

In [ ]:
target_test = label_encoder.inverse_transform(model.predict(features_test))
target_test

In [ ]:
result = pd.DataFrame()
result['instance_id'] = data_test['instance_id'].astype('Int64')
result['music_genre'] = target_test
result.head()

In [ ]:
result.to_csv('/kaggle/working/submit.csv', index=False)

# Вывод

К сожалению я не все успел, что хотел. И считаю, что не справился с заданием.
Слишком много тем в этом блоке было не разобрано в теоретической части, которые пришлось изучать самостоятельно. <br/>
У меня на выходе получался очень маленькое значение f1_micro, по сравнению с другими участниками соревнования, и я почему-то решил, что дело в том, что они все используют catboost. Поэтому я очень много времени потратил на изучение catboost, видимо зря. Потому что catboost дал мне тоже малый итог.<br/>
В соревновании я набрал 0.44409 - но это неосознанный результат, который получился в результате экспериментов. Причем Kaggle дал сбой и я потерял результат работы за 3 дня. Как я добился 0.44409 я так и не понял - повторить не успел.<br/>
Последний результат, который я отправил дал 0.36445.<br/>
Возможно надо было разбираться с важностью признаков, или я еще чего-то не понял.<br/>
Дальше просто не хватило времени.<br/>
вопросов больше чем ответов.
<br/><br/>
Что сделал:<br/>
1. Разобрался с дубликатами.<br/>
2. Заполнил пропуски<br/>
3. Проверил признаки на мультиколлинеарность.<br/>
4. Провел работу с дисбалансом<br/>
5. Методом кроссвалидации проверил 4 модели. Немного разобрался с catboost.<br/>
6. Получил на тренировочной выборке 0.649981, но на тестовой этот результат сильно меньше. К сожалению не понял почему так произошло. В чате тоже народ писал, что у них на обучающей выборке метрика больше.<br/>
7. На самом деле получил много удовольствия от участия в соревновании. Оно подсветило много областей для дальнейшего изучения.
